In [5]:
import numpy as np
def update_zj_cj(new_A,new_B,cb,n_variables,n_constraints):
        tableau = [row[:] for row in new_A]
        transpose_A = list(map(list, zip(*tableau)))
        zj = [np.dot(cb, transpose_A[j]) for j in range(n_variables + n_constraints)]
        cj = [new_B[i] - zj[i] for i in range(n_variables + n_constraints)]
        return zj, cj
def simplex(objective_fun, A, B, type):
    n_constraints = len(A)
    n_variables = len(objective_fun)
    cb = [0] * n_constraints  
    basic = list(range(n_variables, n_variables + n_constraints))  
    new_A = []
    new_B = objective_fun + [0] * n_constraints  

    for i, row in enumerate(A):
        new_row = row.copy()
        identity_part = [1 if j == i else 0 for j in range(n_constraints)]
        new_row.extend(identity_part)
        new_row.append(B[i])
        new_A.append(new_row)

    print("Initial tableau:")
    for row in new_A:
        print(row)
    zj, cj = update_zj_cj(new_A,new_B,cb,n_variables,n_constraints)
    print(f"\nInitial Cj: {cj}")
    print(f"Initial basic variables: {basic}")
    print(f"Initial cb: {cb}")

    iteration = 0
    max_iterations = 100

    if type.lower() == 'maximize':
        while max(cj) > 1e-10 and iteration < max_iterations:  
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")
            key_column = np.argmax(cj)
            print(f"Entering variable (key column): {key_column}")
            possible_rows = [i for i in range(n_constraints) if new_A[i][key_column] > 1e-10]
            if not possible_rows:
                print("Problem is unbounded.")
                return None
            ratios = [abs(new_A[i][-1] / new_A[i][key_column]) for i in possible_rows]
            print(f"Possible rows: {possible_rows}")
            print(f"Ratios: {ratios}")
            min_ratio_idx = np.argmin(ratios)
            key_row = possible_rows[min_ratio_idx]
            print(f"Leaving variable row (key row): {key_row}")
            pivot_element = new_A[key_row][key_column]
            print(f"Pivot element: {pivot_element}")
            if abs(pivot_element) < 1e-10:
                print("Division by zero encountered (pivot is zero).")
                return None

            for j in range(len(new_A[key_row])):
                new_A[key_row][j] /= pivot_element

            for i in range(n_constraints):
                if i != key_row:
                    factor = new_A[i][key_column]
                    for j in range(len(new_A[i])):
                        new_A[i][j] -= factor * new_A[key_row][j]

            old_basic = basic[key_row]
            basic[key_row] = key_column
            cb[key_row] = new_B[key_column]

            print(f"Variable {old_basic} leaves, variable {key_column} enters")
            print(f"Updated basic variables: {basic}")
            print(f"Updated cb: {cb}")

            zj, cj = update_zj_cj(new_A,new_B,cb,n_variables,n_constraints)
            print(f"Updated Cj: {[round(c, 6) for c in cj]}")
            print(f"Updated Zj: {[round(z, 6) for z in zj]}")
            print("Current tableau:")
            for i, row in enumerate(new_A):
                print(f"Row {i}: {[round(x, 6) for x in row]}")

            bj = [row[-1] for row in new_A]
            if any(b < -1e-10 for b in bj):
                print("Warning: Infeasible solution detected (negative bj).")

    elif type.lower() == 'minimize':
        while min(cj) < -1e-10 and iteration < max_iterations:
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")

            key_column = np.argmin(cj)
            print(f"Entering variable (key column): {key_column}")

            possible_rows = [i for i in range(n_constraints) if new_A[i][key_column] > 1e-10]

            if not possible_rows:
                print("Problem is unbounded.")
                return None

            ratios = [abs(new_A[i][-1] / new_A[i][key_column]) for i in possible_rows]
            min_ratio_idx = np.argmin(ratios)
            key_row = possible_rows[min_ratio_idx]

            pivot_element = new_A[key_row][key_column]

            if abs(pivot_element) < 1e-10:
                print("Division by zero encountered.")
                return None

            for j in range(len(new_A[key_row])):
                new_A[key_row][j] /= pivot_element

            for i in range(n_constraints):
                if i != key_row:
                    factor = new_A[i][key_column]
                    for j in range(len(new_A[i])):
                        new_A[i][j] -= factor * new_A[key_row][j]

            basic[key_row] = key_column
            cb[key_row] = new_B[key_column]

            zj, cj = update_zj_cj(new_A,new_B,cb,n_variables,n_constraints)
    else:
        print("Invalid optimization type. Use 'maximize' or 'minimize'.")
        return None

    if iteration >= max_iterations:
        print("Maximum iterations reached!")
        return None

    print(f"\n--- Final Solution (after {iteration} iterations) ---")
    x = [0.0] * n_variables
    for i in range(n_constraints):
        col = basic[i]
        if col < n_variables:  
            x[col] = new_A[i][-1]

    bj = [row[-1] for row in new_A]
    optimal_value = np.dot(cb, bj)

    print(f"Basic variables: {basic}")
    print(f"Solution: {[round(xi, 6) for xi in x]}")
    print(f"Optimal value: {round(optimal_value, 6)}")

    return x, optimal_value

# Test the function
print("Testing Simplex Algorithm")
print("========================")
# objective_fun = [100, 500, 600,700,800]
# A = [[10,20,30,40,50],
#      [50, 60, 10,10,5],
#      [45, 68, 14,74,8],
#      [26,65,48,13,17],
#      [100,13,41,23,15]]
# B = [10000, 20000, 50000,40000,75000]

# objective_fun=[100,10,1]
# A=[[1,0,0],[20,1,0],[200,20,1]]
# B=[1,100,10000]

objective_fun = [3,5]
A=[[1,0],[0,2],[3,2]]
B=[4,12,18]
result = simplex(objective_fun, A, B, 'maximize')

Testing Simplex Algorithm
Initial tableau:
[1, 0, 1, 0, 0, 4]
[0, 2, 0, 1, 0, 12]
[3, 2, 0, 0, 1, 18]

Initial Cj: [np.int64(3), np.int64(5), np.int64(0), np.int64(0), np.int64(0)]
Initial basic variables: [2, 3, 4]
Initial cb: [0, 0, 0]

--- Iteration 1 ---
Entering variable (key column): 1
Possible rows: [1, 2]
Ratios: [6.0, 9.0]
Leaving variable row (key row): 1
Pivot element: 2
Variable 3 leaves, variable 1 enters
Updated basic variables: [2, np.int64(1), 4]
Updated cb: [0, 5, 0]
Updated Cj: [np.float64(3.0), np.float64(0.0), np.float64(0.0), np.float64(-2.5), np.float64(0.0)]
Updated Zj: [np.float64(0.0), np.float64(5.0), np.float64(0.0), np.float64(2.5), np.float64(0.0)]
Current tableau:
Row 0: [1.0, 0.0, 1.0, 0.0, 0.0, 4.0]
Row 1: [0.0, 1.0, 0.0, 0.5, 0.0, 6.0]
Row 2: [3.0, 0.0, 0.0, -1.0, 1.0, 6.0]

--- Iteration 2 ---
Entering variable (key column): 0
Possible rows: [0, 2]
Ratios: [4.0, 2.0]
Leaving variable row (key row): 2
Pivot element: 3.0
Variable 4 leaves, variable 0 ent

In [2]:
import numpy as np

def convert_to_dual(objective_fun, A, B):
    """
    Converts primal LP to dual LP in standard form for maximization with <= constraints.
    """
    objective_fun_dual = [-b for b in B]
    A_transpose = list(map(list, zip(*A)))
    A_dual = [[-a for a in row] for row in A_transpose]
    B_dual = [-c for c in objective_fun]
    return objective_fun_dual, A_dual, B_dual

In [6]:
import numpy as np
def update_zj_cj_dual(new_A,new_B,cb_matrix,n_variables,n_constraints):
        tableau_without_rhs = [row[:] for row in new_A]
        transpose_A = list(map(list, zip(*tableau_without_rhs)))
        zj = [np.dot(cb_matrix, transpose_A[j]) for j in range(n_variables + n_constraints)]
        cj = [zj[i] - new_B[i]  for i in range(n_variables + n_constraints)]
        return zj, cj
def dual_simplex(objective_fun,A,B,type):
  # objective_fun, A, B = convert_to_dual(objective_fun, A, B)
  n_constraints = len(A)
  n_variables = len(objective_fun)
  cb = [0]*n_constraints
  basic = list(range(n_variables,n_variables+n_constraints))
  new_A = []
  new_B = objective_fun + [0]*n_constraints
  for i , row in enumerate(A):
    new_row = row.copy()
    identity_part = [1 if j == i else 0 for j in range(n_constraints)]
    new_row.extend(identity_part)
    new_row.append(B[i])
    new_A.append(new_row)
  print("New A matrix:")
  for row in new_A:
    print(row)
  zj, cj = update_zj_cj_dual(new_A,new_B,cb,n_variables,n_constraints)
  print(f"\nInitial Cj: {cj}")
  print(f"Initial basic variables: {basic}")
  print(f"Initial cb: {cb}")
  print(f"Initial zj: {zj}")
  bj = [row[-1] for row in new_A]
  print(f"Initial Bj: {bj}")

  iteration = 0
  max_iterations = 100
  if type.lower() == 'maximize':
          print('-'*50)
          while(min(bj)<=0 and min(cj)<=0 and iteration < max_iterations):
            iteration +=1
            print(f"\n--- Iteration {iteration} ---")
            key_row = np.argmin(bj)
            print(f"Entering variable (key row): {key_row}")
            possible_rows = [i for i in range(n_constraints+n_variables) if new_A[key_row][i] < 0]
            if not possible_rows:
                print("Problem is unbounded.")
                return None
            ratios = [cj[i]/new_A[key_row][i] for i in possible_rows]
            max_ratio_idx = np.argmax(ratios)
            key_column = possible_rows[max_ratio_idx]
            print(f"Leaving variable row (key column): {key_column}")
            pivot_element = new_A[key_row][key_column]
            if(abs(pivot_element)<1e-10):
              print("Division by zero encountered (pivot is zero).")
              return None
            for j in range(len(new_A[key_row])):
                new_A[key_row][j]/=pivot_element
            for k in range(n_constraints):
              if(k!=key_row):
                factor = new_A[k][key_column]
                for h in range(len(new_A[k])):
                   new_A[k][h] -=factor * new_A[key_row][h]
            old_basic = basic[key_row]
            basic[key_row] = key_column
            cb[key_row] = new_B[key_column]
            print(f"Variable {old_basic} leaves, variable {key_column} enters")
            print(f"Updated basic variables: {basic}")
            print(f"Updated cb_matrix: {cb}")

            zj, cj = update_zj_cj_dual(new_A,new_B,cb,n_variables,n_constraints)
            print(f"Updated Cj: {[round(c, 6) for c in cj]}")

            print("Current tableau:")
            for i, row in enumerate(new_A):
                print(f"Row {i}: {[round(x, 6) for x in row]}")

            bj = [row[-1] for row in new_A]
            if any(b < -1e-10 for b in bj):
                print("Warning: Infeasible solution detected (negative bj).")

  print(f"\n--- Final Solution (after {iteration} iterations) ---")
  x = [0.0] * n_variables
  for i in range(n_constraints):
        col = basic[i]
        if col < n_variables:  
            x[col] = new_A[i][-1]

  bj = [row[-1] for row in new_A]
  optimal_value = np.dot(cb, bj)

  print(f"Basic variables: {basic}")
  print(f"Solution: {[round(xi, 6) for xi in x]}")
  print(f"Optimal value: {round(optimal_value, 6)}")

  return x, optimal_value
objective_fun = [-5,-6]
A=[[-1,-1],[-4,-1]]
B=[-2,-4]

# objective_fun = [ -2, -3]
# A=[[-1,-1],[2,1],[1,2]]
# B=[-2,10,8]

# objective_fun = [-4,-12,-18]
# A=[[-1,0,-3],[0,-2,-2]]
# B=[-3,-5]

# objective_fun = [2,3]
# A=[[-1,1],[1,3],[1,0]]
# B=[5,35,20]

dual_simplex(objective_fun,A,B,'maximize')

New A matrix:
[-1, -1, 1, 0, -2]
[-4, -1, 0, 1, -4]

Initial Cj: [np.int64(5), np.int64(6), np.int64(0), np.int64(0)]
Initial basic variables: [2, 3]
Initial cb: [0, 0]
Initial zj: [np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Initial Bj: [-2, -4]
--------------------------------------------------

--- Iteration 1 ---
Entering variable (key row): 1
Leaving variable row (key column): 0
Variable 3 leaves, variable 0 enters
Updated basic variables: [2, 0]
Updated cb_matrix: [0, -5]
Updated Cj: [np.float64(0.0), np.float64(4.75), np.float64(0.0), np.float64(1.25)]
Current tableau:
Row 0: [0.0, -0.75, 1.0, -0.25, -1.0]
Row 1: [1.0, 0.25, -0.0, -0.25, 1.0]

--- Iteration 2 ---
Entering variable (key row): 0
Leaving variable row (key column): 3
Variable 2 leaves, variable 3 enters
Updated basic variables: [3, 0]
Updated cb_matrix: [0, -5]
Updated Cj: [np.float64(0.0), np.float64(1.0), np.float64(5.0), np.float64(0.0)]
Current tableau:
Row 0: [-0.0, 3.0, -4.0, 1.0, 4.0]
Row 1: [1.0, 1.0

([2.0, 0.0], np.float64(-10.0))